In [1]:
import pandas as pd
import ast

In [2]:
game = pd.read_csv('C:/myprojects/KPL-Project/data/Seasons/2014/rounds.csv')
table = pd.read_csv('C:/myprojects/KPL-Project/data/Seasons/2014/table.csv')

In [3]:

game['Date'] = pd.to_datetime(game['Date'])

In [4]:
teamdim = table[['TeamID', 'TeamName', 'NameCode', 'TeamColors']].drop_duplicates().reset_index(drop=True)
teamdim['TeamColors'] = teamdim['TeamColors'].apply(ast.literal_eval)
teamdim[['PrimaryColor','SecondaryColor']] = pd.DataFrame(teamdim['TeamColors'].tolist(), index=teamdim.index)
teamdim.drop(columns=['TeamColors'], inplace=True)
teamdim.set_index('TeamID')

,TeamName,NameCode,PrimaryColor,SecondaryColor
TeamID,,,,
134306,Gor Mahia FC,GOR,#008000,#000000
138488,Sofapaka,SOF,#ffffff,#0000ff
138498,Ulinzi Stars,ULS,#ff0000,#ffffff
138496,Tusker,TUS,#ffdf00,#000000
138476,Chemelil Sugar,CSU,#ffff00,#000000
76433,AFC Leopards SC,LDS,#ffffff,#0000ff
138484,Muhoroni Youth FC,MUY,#374df5,#374df5
138490,Sony Sugar,SSU,#008633,#ffffff
138492,Thika United,TUN,#374df5,#374df5


In [5]:
seasondim = game[['Season']].drop_duplicates().reset_index(drop=True)
seasondim.rename(columns={'Season': 'SeasonID'}, inplace=True)
seasondim['SeasonName'] = 'SSN-' + str(game['Season'][0])
seasondim.set_index('SeasonID')


,SeasonName
SeasonID,
2014,SSN-2014


In [6]:
tournaments = {
    'Kenyan Premier League': 1,
    'Kenyan Super Cup': 2
}
tournamentdim = game[['Tournament']].reset_index(drop=True).drop_duplicates()
tournamentdim['TournamentID'] = tournamentdim['Tournament'].map(tournaments)
tournamentdim.set_index('TournamentID')

,Tournament
TournamentID,
1,Kenyan Premier League


In [8]:
melt_game = game.melt(id_vars=['MatchID', 'MatchCustomID', 'Round', 'Date','StartTime','Season',
                                'MatchStatus', 'HomeScoreFT', 'AwayScoreFT', 'Tournament'],
                      value_vars=['HomeTeamID', 'AwayTeamID'],
                      var_name='TeamType',
                      value_name='TeamID').rename(columns={'Season': 'SeasonID'})
facttable = melt_game.merge(teamdim, on='TeamID').merge(seasondim, on='SeasonID').merge(tournamentdim, on='Tournament')
facttable.head(15)

,MatchID,MatchCustomID,Round,Date,StartTime,SeasonID,MatchStatus,HomeScoreFT,AwayScoreFT,Tournament,TeamType,TeamID,TeamName,NameCode,PrimaryColor,SecondaryColor,SeasonName,TournamentID
0,5022474,JufbsNufb,1,2014-02-15,15:00,2014,Finished,1,1,Kenyan Premier League,HomeTeamID,138484,Muhoroni Youth FC,MUY,#374df5,#374df5,SSN-2014,1
1,5022472,PufbsUufb,1,2014-02-15,15:00,2014,Finished,3,3,Kenyan Premier League,HomeTeamID,138490,Sony Sugar,SSU,#008633,#ffffff,SSN-2014,1
2,5022476,BufbsYufb,1,2014-02-15,15:00,2014,Finished,1,1,Kenyan Premier League,HomeTeamID,138498,Ulinzi Stars,ULS,#ff0000,#ffffff,SSN-2014,1
3,5022470,Hufbsavfb,1,2014-02-15,15:00,2014,Finished,1,1,Kenyan Premier League,HomeTeamID,138500,Western Stima FC,WST,#ffff00,#0000ff,SSN-2014,1
4,5022478,zufbsFufb,1,2014-02-16,15:00,2014,Finished,2,2,Kenyan Premier League,HomeTeamID,138474,Bandari FC,BAN,#3050c6,#ffffff,SSN-2014,1
5,5022480,RufbsWufb,1,2014-02-16,15:00,2014,Finished,1,1,Kenyan Premier League,HomeTeamID,138492,Thika United,TUN,#374df5,#374df5,SSN-2014,1
6,5022484,IDFsDufb,1,2014-02-19,15:00,2014,Finished,1,1,Kenyan Premier League,HomeTeamID,138478,Kenya Commercial Bank,KCB,#00dd00,#004165,SSN-2014,1
7,5022482,gLdbsLufb,1,2014-02-20,15:00,2014,Finished,2,2,Kenyan Premier League,HomeTeamID,134306,Gor Mahia FC,GOR,#008000,#000000,SSN-2014,1
8,5065702,DufbsFufb,2,2014-02-22,14:00,2014,Finished,1,1,Kenyan Premier League,HomeTeamID,138480,Kenya Rev. Authority,KAU,#374df5,#374df5,SSN-2014,1
9,5065704,BufbsJufb,2,2014-02-22,15:00,2014,Finished,2,2,Kenyan Premier League,HomeTeamID,138476,Chemelil Sugar,CSU,#ffff00,#000000,SSN-2014,1
